In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rc
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import re
from wordcloud import WordCloud, STOPWORDS

rc('font', family='Malgun Gothic')
rc('axes', unicode_minus=False)

In [ ]:
import csv
f = open('data_all.csv', 'r', encoding='cp949')
lines = csv.reader(f)
header = next(lines)
print(header)
f.close()

['brand', 'date', 'rate', 'id', 'skin_type', 'select_1_title', 'select_1_content', 'select_2_title', 'select_2_content', 'select_3_title', 'select_3_content', 'txt']


In [ ]:
# 데이터 불러오기
review = pd.read_csv('data_all.csv', encoding='cp949')

In [ ]:
# 컬럼 제거
review = review.drop(columns=['date', 'id', 'skin_type','select_1_title','select_2_title','select_3_title'])

In [ ]:
# 컬럼 리네임
review.rename({'brand':'브랜드', 'rate': '평점', 'select_1_content':'피부타입',
               'select_2_content':'피부고민','select_3_content':'자극도', 'txt':'리뷰' }, axis=1, inplace=True)
review.columns

Index(['브랜드', '평점', '피부타입', '피부고민', '자극도', '리뷰'], dtype='object')

In [ ]:
review.head()

,브랜드,평점,피부타입,피부고민,자극도,리뷰
0,닥터지,5점만점에 4점,복합성에 좋아요,진정에 좋아요,보통이에요,컵 받고싶어서 샀어요\n크림은 튜브형태러 내줄 생각은 없는건지 궁금해요
1,닥터지,5점만점에 5점,건성에 좋아요,진정에 좋아요,자극없이 순해요,닥터지는 사계절 내내 잘 사용하고잇어요 ㅎㅎㅎ\n\n물론 건성피부에게 겨울에는 얼굴...
2,닥터지,5점만점에 5점,복합성에 좋아요,진정에 좋아요,보통이에요,아빠 사드렸는데 좋아해요 근데 바르는게 쫌 별로긴 한데 투브타입으로 나오면 더 좋을 듯
3,닥터지,5점만점에 5점,건성에 좋아요,보습에 좋아요,자극없이 순해요,국민 수분 크림이라 그런지 모든 피부타입에도 잘 맞으면서 무난한 아이템이에요 당황이...
4,닥터지,5점만점에 4점,복합성에 좋아요,보습에 좋아요,자극없이 순해요,"- 여름용 수분크림이 필요해서 산 제품이에요.\n- 가격은 세일가로 28,800원 ..."


전처리 작업

In [ ]:
# '\n' 제거
def remove_newline(text):
    return re.sub(r'\n', ' ', text)

review['리뷰'] = review['리뷰'].apply(remove_newline)

In [ ]:
# 평점 중 점수 추출
def score_conversion(text):
    score_match = re.search(r'(\d+)점$', text) # 숫자 다음 '점'이 있는 패턴 찾기, ex) 5점만점에 4점
    if score_match:
        return score_match.group(1) # 추출된 숫자 반환(score_match에서 괄호로 묶인 첫 번째 그룹(숫자) 반환)
    else:
        return None

review['평점'] = review['평점'].apply(score_conversion)

In [ ]:
# 구두점 제거
from string import punctuation

def repreprocessing(text):
    return ''.join(word for word in text if word not in punctuation or word == '+')

review['리뷰'] = review['리뷰'].apply(repreprocessing)

In [ ]:
# 하트 및 '+'를 제외한 구두점 제거
def cleanText(text):
    text = re.sub('[-=,#/\?:;^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》❤️♥️♡]', '', text)
    return text

review['리뷰'] = review['리뷰'].apply(cleanText)

In [ ]:
# 자음 반복 제거
def cleanText2(text):

    text = re.sub('[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]', '', text)
    return text


review['리뷰'] = review['리뷰'].apply(cleanText2)

In [ ]:
# 전처리 완료 후 모습
review.head()

,브랜드,평점,피부타입,피부고민,자극도,리뷰
0,닥터지,4,복합성에 좋아요,진정에 좋아요,보통이에요,컵 받고싶어서 샀어요 크림은 튜브형태러 내줄 생각은 없는건지 궁금해요
1,닥터지,5,건성에 좋아요,진정에 좋아요,자극없이 순해요,닥터지는 사계절 내내 잘 사용하고잇어요 물론 건성피부에게 겨울에는 얼굴당김잇어서...
2,닥터지,5,복합성에 좋아요,진정에 좋아요,보통이에요,아빠 사드렸는데 좋아해요 근데 바르는게 쫌 별로긴 한데 투브타입으로 나오면 더 좋을 듯
3,닥터지,5,건성에 좋아요,보습에 좋아요,자극없이 순해요,국민 수분 크림이라 그런지 모든 피부타입에도 잘 맞으면서 무난한 아이템이에요 당황이...
4,닥터지,4,복합성에 좋아요,보습에 좋아요,자극없이 순해요,여름용 수분크림이 필요해서 산 제품이에요 가격은 세일가로 28800원 했어요사실...


In [ ]:
# 데이터 개수 확인
review.shape

(5911, 6)

In [ ]:
# null 확인
review.isnull().sum()

브랜드     0
평점      0
피부타입    0
피부고민    0
자극도     0
리뷰      0
dtype: int64